# Thompson's Incomplete Rust Notebook

## Closures

### Cloning without renaming

TODO

## Non-const static initializers

### Pure standard library

The once_cell crate has been merged into the standard library. This allows us to create macros to cover common use-cases such as JIT compiling a regex. This macro also allows a more fluid approach to using regular expression (because the fact we are pre-compiling it is hidden at the point of use).

In [24]:
:dep regex

/// Compile a regex once and at the point-of-first use.
///
/// Takes string literal and return an expression that evaluates to &'static Regex
///
/// It is a standard library port of an example provides in the once_cell crate:
/// https://docs.rs/once_cell/latest/once_cell/#lazily-compiled-regex
///
/// # Examples
///
/// ```
/// let ln = "[ 0.873163] usb 10-1.1.2.4: SerialNumber: 0x0001";
/// let match = regex!(r"^\[ *([0-9.]+)] usb ([0-9-.]+): (.*)$").captures(ln)
/// ```
macro_rules! regex {
    ($re:literal $(,)?) => {{
        static RE: std::sync::OnceLock<regex::Regex> = std::sync::OnceLock::new();
        RE.get_or_init(|| regex::Regex::new($re).unwrap())
    }};
}

#### Examples

In [26]:
let ln = "[ 0.873163] usb 10-1.1.2.4: SerialNumber: 0x0001";

regex!(r"^\[ *([0-9.]+)] usb ([0-9-.]+): (.*)$").captures(ln)

Some(Captures({0: 0..48/"[ 0.873163] usb 10-1.1.2.4: SerialNumber: 0x0001", 1: 2..10/"0.873163", 2: 16..26/"10-1.1.2.4", 3: 28..48/"SerialNumber: 0x0001"}))

### crate: lazy_static

TODO

### crate: once_cell

The [once_cell](https://docs.rs/once_cell) crate has been partially merged into the standard library. The low-level plumbing is present in stable but the [easy to use utilitity code](https://doc.rust-lang.org/std/cell/struct.LazyCell.html) is [still unstable](https://github.com/rust-lang/rust/issues/109736).

The utility code implements a similar use case as the lazy_static!() macro but without the macro! Note that the example below uses a regex since it is easy to read. However be aware that there are other, more elegant ways, to handle regular expressions using OnceCell/OnceLock (see above).

In [4]:
:dep regex
:dep once_cell

use once_cell::sync::Lazy;

static RE: Lazy<regex::Regex> = Lazy::new(|| {
    regex::Regex::new(r"^\[ *([0-9.]+)] usb ([0-9-.]+): (.*)$").unwrap()
});

#### Examples

In [8]:
RE.captures("[ 0.873163] usb 10-1.1.2.4: SerialNumber: 0x0001")

Some(Captures({0: 0..48/"[ 0.873163] usb 10-1.1.2.4: SerialNumber: 0x0001", 1: 2..10/"0.873163", 2: 16..26/"10-1.1.2.4", 3: 28..48/"SerialNumber: 0x0001"}))

### crate: static_init

## Interesting templates

### Application wide CLI arguments

Using crates such as static_init (or similar less heavily optimized crates such as lazy_static) we are able to parse the arguments on first access and access them via a shared immutable global variable anywhere in the application.

The jury is still out on whether this is a pattern or an anti-pattern. I generally only use lazy_static and friends for const-like data such as pre-compiled regular expressions. Sometimes it iss hard to decide if command line arguments, which should never change for the duration of a program, are const-like or not! Either way, it certainly needs to be used with taste and restraint.

In [2]:
:dep clap = { features = ["derive"] }
:dep static_init

use clap::Parser;
use static_init::dynamic;

#[derive(Parser)]
#[command(author, version, about, long_about = None)]
pub struct Args {
    /// Turn debugging information on
    #[arg(short, long, action = clap::ArgAction::Count)]
    pub verbose: u8,
}

// If you prefer lazy_static ARGS could have been spelled:
//   lazy_static! {
//       pub static ref ARGS: Args = Args::parse();
//   }
#[dynamic]
pub static ARGS: Args = Args::parse();

fn main() {
    println!("{}", ARGS.verbose);
}

### Regex pattern extraction

This is a great little pattern for extracting matching lines of text and robustly extracting fields from them. Fields can be fully parsed or returned as raw string slices and, in the later case, are zerop-copy.

This code comes for the [Linaro Developer Services keynote](https://resources.linaro.org/en/resource/6ttLWXTBe7VazbNBK1g1QW) at Linaro Connect 2023. The presentation consists of multiple speakers covering a variety of topics. The Rust material starts at 23:28.

In [3]:
:dep lazy_static
:dep regex

use lazy_static::lazy_static;
use regex::Regex;

fn parse_usb_msg(ln: &str) -> Option<(f64, &str, &str)> {
  lazy_static! {
    static ref RE: Regex = Regex::new(
      //  [   0.873163] usb 10-1.1.2.4: SerialNumber: 0x0001
      r"^\[ *([0-9.]+)] usb ([0-9-.]+): (.*)$").unwrap();
  }

  let fields = RE.captures(ln)?;
  let tstamp = fields
      .get(1)?
      .as_str()
      .parse()
      .ok()?;
  let path = fields.get(2)?.as_str();
  let msg = fields.get(3)?.as_str();

  Some((tstamp, path, msg))
}

#### Examples

In [4]:
parse_usb_msg("[ 0.873163] usb 10-1.1.2.4: SerialNumber: 0x0001")

Some((0.873163, "10-1.1.2.4", "SerialNumber: 0x0001"))

In [5]:
parse_usb_msg("[ 0.873163] usb XX-X.X.X.X: SerialNumber: 0x0001")

None